## TP Linear Model : Binary Classification

In [ ]:
# builtin
import os, warnings

# data
import pandas as pd
import numpy as np

# preprocessing
from sklearn.preprocessing import LabelEncoder
# metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix
# estimators
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
# model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# visualisation
import matplotlib.pyplot as plt

In [ ]:
# warnings.filterwarnings('ignore')
warnings.filterwarnings(action="once")

In [ ]:
X_train, X_test, y_train, y_test = pd.DataFrame({})

### 3. Modelisation

#### 3.1 Dummy classifier

In [ ]:
estimator = DummyClassifier(strategy="most_frequent")
estimator.fit(X_train, y_train)

In [ ]:
y_pred = estimator.predict(X_test)
y_pred

In [ ]:
pd.Series(y_pred).values_counts() # transform the numpy array into a Pandas Series so that we can use the values count method

In [ ]:
y_test.value_counts(normalize=True)

In [ ]:
y_test.values

In [ ]:
def score(estimator):
    """
    Computes and prints train score and test score.
    :param estimator:
    :return:
    """
    tr_score = estimator.score(X_train, y_train).round(4)
    te_score = estimator.score(X_test, y_test).round(4)

    print(f"score train : {tr_score} score test : {te_score} ")

In [ ]:
score(estimator)

In [ ]:
def confusion(y_test, y_pred):
    """
    Displays a fancy confusion matrix
    :param y_test:
    :param y_pred:
    :return:
    """
    mat = confusion_matrix(y_test, y_pred) # a numpy array
    mat = pd.DataFrame(mat)
    mat.columns = [f"pred_{i}" for i in mat.columns]
    mat.index = [f"test_{i}" for i in mat.index]

    return mat

In [ ]:
confusion(y_pred, y_test)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate, color="red", label= "AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.axis("tight")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

#### 3.2 Simple Logistic regression & Linear SVM

##### 3.2.1 Logistic Regression


In [ ]:
estimator = LogisticRegression(solver="liblinear")
estimator.fit(X_train, y_train)

In [ ]:
y_pred = estimator.predict(X_test)
y_pred

In [ ]:
y_prob = estimator.predict_proba(X_test).round(2)
y_prob # la confiance / proba d'appartenance à telle ou telle classe

In [ ]:
score(estimator)
confusion(y_test, y_pred)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate).round(2)
print(roc_auc)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate, color="red", label= "AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.axis("tight")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

##### 3.2.2 SVM

In [ ]:
estimator = LinearSVC()
estimator.fit(X_train, y_train)

In [ ]:
y_pred = estimator.predict(X_test)
y_pred

In [4]:
# linear svc pas de y_proba comme méthode

In [ ]:
score(estimator)
confusion(y_test, y_pred)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate).round(2)
print(roc_auc)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate, color="red", label= "AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.axis("tight")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

#### 3.3 Finding Hyper-parameters

##### 3.3.1 Logistic Regression

In [ ]:
estimator = LogisticRegression()

params = {
    'C': np.logspace(-3, 3, 7),
    'penalty': ['l1', 'l2'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear']
}

In [ ]:
grid = GridSearchCV(estimator,
                    params,
                    cv=10,
                    n_jobs=-1,
                    return_train_score=True,
                    verbose=1)

grid.fit(X_train, y_train)
# cv=10 : donc 10 folds == train on 10 train set => moyenne sur les 10 et std deviation donc idée ==> idée de la variance
# verbose = qtt info renvoyé par la grid

# params :dict de string et chq param testé doit etre un itérable (list)
# params : clé => valeur (list for example)

In [ ]:
best_params = grid.best_params_
best_params

In [ ]:
res = grid.cv_results_
res = pd.DataFrame(res)
res

In [ ]:
cols = [i for i in res.columns if "split" not in i]
cols # so that we will have an average of the scores

In [ ]:
res = res[cols]
res = res.sort_values("rank_test_score")
res.head(10)

In [ ]:
def rezultize(grid):
    """
    Make a fancy df from grid.cv_results
    :param grid:
    :return:
    """
    res = grid.cv_results_
    res = pd.DataFrame(res)
    cols = [i for i in res.columns if "split" not in i]
    res = res[cols]
    res = res.sort_values("rank_test_score")

    return res

In [ ]:
rezultize(grid).head(10)

Train again the estimator with the best parameters

In [ ]:
# OR automatic operation
estimator = LogisticRegression(**best_params)

In [ ]:
estimator = LogisticRegression(**best_params)
estimator.fit(X_train, y_train) # no folds here
y_pred = estimator.predict(X_test)
y_pred

In [ ]:
score(estimator)
confusion(y_test, y_pred)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate).round(2)
print(roc_auc)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate, color="red", label= "AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.axis("tight")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

Plus de confiance dans le model comme pls entrainement faits
progres 1% ?
avant marge 5% erreur
progression 1/5 = 20% erreur
?
reduit l'erreur de 20%

##### 3.3.2 SVM

In [ ]:
estimator = LinearSVC()
params = {'C': np.logspace(-3, 3, 7)}
# 7 * 10 cv = 70 fits

https://openclassrooms.com/fr/courses/4444646-entrainez-un-modele-predictif-lineaire/6774321-entrainez-vous-a-classer-automatiquement-des-feuilles-d-arbres TRY TO DO THIS PROJECT ;)